# 03 - Interactive Viz

## Deadline

Wednesday November 8th, 2017 at 11:59PM

## Important Notes

- Make sure you push on GitHub your Notebook with all the cells already evaluated
- Note that maps do not render in a standard Github environment : you should export them to HTML and link them in your notebook.
- Remember that `.csv` is not the only data format. Though they might require additional processing, some formats provide better encoding support.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
- Please write all your comments in English, and use meaningful variable names in your code

## Background

In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Unemployment rates are major economic metrics and a matter of concern for governments around the world. Though its definition may seem straightforward at first glance (usually defined as the number of unemployed people divided by the active population), it can be tricky to define consistently. For example, one must define what exactly unemployed means : looking for a job ? Having declared their unemployment ? Currently without a job ? Should students or recent graduates be included ? We could also wonder what the active population is : everyone in an age category (e.g. `16-64`) ? Anyone interested by finding a job ? Though these questions may seem subtle, they can have a large impact on the interpretation of the results : `3%` unemployment doesn't mean much if we don't know who is included in this percentage. 

In this homework you will be dealing with two different datasets from the statistics offices of the European commission ([eurostat](http://ec.europa.eu/eurostat/data/database)) and the Swiss Confederation ([amstat](https://www.amstat.ch)). They provide a variety of datasets with plenty of information on many different statistics and demographics at their respective scales. Unfortunately, as is often the case is data analysis, these websites are not always straightforward to navigate. They may include a lot of obscure categories, not always be translated into your native language, have strange link structures, â€¦ Navigating this complexity is part of a data scientists' job : you will have to use a few tricks to get the right data for this homework.

For the visualization part, install [Folium](https://github.com/python-visualization/folium) (*HINT*: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!). Folium's `README` comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find two `.topojson` files, containing the geo-coordinates of 

- European countries (*liberal definition of EU*) (`topojson/europe.topojson.json`, [source](https://github.com/leakyMirror/map-of-europe))
- Swiss cantons (`topojson/ch-cantons.topojson.json`) 

These will be used as an overlay on the Folium maps.

## Assignment

1. Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

2. Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

3. Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

4. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [Röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

## Task 1

Go to [eurostat](http://ec.europa.eu/eurostat/data/database) and find the following dataset relative to this task:

* EU Countries: the first two have monthly numbers since 2016, and the third has yearly numbers between 1990 and 2016.
 * ei_lmhr_m
 * ei_lmhu_m
 * tsdec450
* Candidate and potential candidates countries:
 * cpc_siemp
* Eastern Europe and other countries: the first one contains only employment rates.
 * enpr_siemp
 * med_ps421
 * med_ps422

These datasets may have different definition in unemployment rate, so we have to becareful when aggregating them.

To start with, load the unemployment rate in 2015 from ```tsdec450```.

In [2]:
import json
import numpy as np
import pandas as pd
import folium

In [2]:
# load data
df = pd.ExcelFile('Data/Eurostat_Table_tsdec450FlagDesc_e44a024d-526c-4dc1-8505-6438fed7c9c7.xls').parse('Sheet0')
# select columns: country name and 2015
df = df[['Total unemployment rate', 'Unnamed: 51']]
df.columns = ['Country', 'Percentage']
# select country rows (not include Switzerland)
eu_rate = df.loc[np.r_[7:37,38]].reset_index(drop=True)
# get country id and name
eu_rate['Country_id'] = eu_rate.Country.apply(lambda s: s.split(':')[0])
eu_rate['Country_name'] = eu_rate.Country.apply(lambda s: s.split(':')[1])
eu_rate = eu_rate[['Country_name', 'Percentage']]
eu_rate.head()

,Country_name,Percentage
0,Belgium,8.5
1,Bulgaria,9.2
2,Czech Republic,5.1
3,Denmark,6.2
4,Germany,4.6


Because folium has more render options for geojson file, we decide to transform topojson file to geojson file. We use [this website](http://jeffpaine.github.io/geojson-topojson/) to do the transformation.


In [3]:
# create the map
eu_map = folium.Map(location=[57.51833,20.6122144], tiles='cartodbpositron',zoom_start=3)
# load boundaries of countries
geo_json_data = json.load(open('topojson/europe.geojson.json'))

In [4]:
eu_map.choropleth(geo_data = geo_json_data,
                  data = eu_rate,
                  columns = ['Country_name', 'Percentage'],
                  # set names as keys because some ids don't match :|
                  key_on = 'feature.properties.NAME',
                  highlight = True,
                  fill_color = 'BuPu',
                  fill_opacity = 0.7,
                  line_opacity = 0.2,
                  legend_name = 'Unemployment rate in the EU (%)')

In [5]:
eu_map

In [6]:
# display the map using html (may or may not work)
from IPython.core.display import HTML
url = 'EU_unemployment_rate.html'
eu_map.save(url)
HTML('<iframe src=' + url + ' width=100% height=500></iframe>')

### Todo
1. Select the data we want:
 * Which countries? Only EU or all the 51 countries?
 * Which number? We don't have too many monthly numbers. Some countries have a lot of missing data, e.g. Switzerland.
2. Combine the numbers with topojson.
3. Plot the map. Adjust the visualization.
4. How to indicate NaNs (if any)?
5. "Compare Switzerland's unemployment rate to that of the rest of Europe."
6. Find a way to present our homework.

## Task 2

In [4]:
# load data
df = pd.read_excel('Data/2_1 Taux de chômage.xlsx', header=[5, 6])
df.drop('Mesures', axis=1, level=1, inplace=True)
df

Canton                          Janvier 2017                    \
                             Taux de chômage Chômeurs inscrits   
Zurich                                   3.9             32387   
Berne                                    3.0             16954   
Lucerne                                  2.2              4985   
Uri                                      1.5               297   
Schwyz                                   2.0              1794   
Obwald                                   1.0               217   
Nidwald                                  1.3               305   
Glaris                                   2.5               567   
Zoug                                     2.6              1756   
Fribourg                                 3.2              5215   
Soleure                                  3.2              4744   
Bâle-Ville                               4.2              4181   
Bâle-Campagne                            3.1              4659   
Schaffhouse                              3.6              1587   
Appenzell Rhodes-Extérieures             2.0               610   
Appenzell Rhodes-Intérieures             1.4               129   
St-Gall                                  2.7              7597   
Grisons                                  1.9              2117   
Argovie                                  3.5             12622   
Thurgovie                                2.6              3967   
Tessin                                   4.0              6757   
Vaud                                     5.2             20672   
Valais                                   5.2              9059   
Neuchâtel                                6.6              6052   
Genève                                   5.7             13306   
Jura                                     5.3              1930   
Total                                    3.7            164466   

Canton                                            \
                             Demandeurs d'emploi   
Zurich                                     39340   
Berne                                      21586   
Lucerne                                     7950   
Uri                                          464   
Schwyz                                      2581   
Obwald                                       378   
Nidwald                                      535   
Glaris                                       876   
Zoug                                        2831   
Fribourg                                    9197   
Soleure                                     7309   
Bâle-Ville                                  5954   
Bâle-Campagne                               6111   
Schaffhouse                                 2653   
Appenzell Rhodes-Extérieures                 978   
Appenzell Rhodes-Intérieures                 171   
St-Gall                                    11939   
Grisons                                     3860   
Argovie                                    17038   
Thurgovie                                   6374   
Tessin                                     10419   
Vaud                                       28019   
Valais                                     12989   
Neuchâtel                                   7712   
Genève                                     16984   
Jura                                        2613   
Total                                     226861   

Canton                                                           Février 2017  \
                             Demandeurs d'emploi non chômeurs Taux de chômage   
Zurich                                                   6953             3.9   
Berne                                                    4632             3.0   
Lucerne                                                  2965             2.2   
Uri                                                       167             1.4   
Schwyz                                                    787             2.0   
Obwald   

### Todo
Almost everything. Just started.